# Convolution Image Net

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from utils.nn_visualization import variable_summaries
from utils.data import init_model_logging
from utils.nn_graph import simple_layer
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import os
import pandas as pd
import matplotlib.pylab as plt

## Load data

In [ ]:
data = input_data.read_data_sets('/data/fashion/', one_hot=True)
img_shape = (28, 28)
class_id2class_name_mapping = {
    0: 'T-shirt/top',
    1: 'Trouser',
    2: 'Pullover',
    3: 'Dress',
    4: 'Coat',
    5: 'Sandal',
    6: 'Shirt',
    7: 'Sneaker',
    8: 'Bag',
    9: 'Ankle boot'}

## Custom Layer Functions

In [ ]:
def conv_layer(name, input_data, 
               conv_filter_shape, pooling_filter_shape, 
               conv_filter_stride=[1, 1, 1, 1], 
               pooling_filter_stride=[1, 1, 1, 1],
               padding='SAME',
               is_training=True):
    w_name = 'w_' + name
    b_name = 'b_' + name
    
    # Variables initialization
    w = tf.get_variable(w_name, conv_filter_shape, initializer=tf.contrib.layers.variance_scaling_initializer())
    bias = tf.get_variable(b_name, initializer=tf.constant_initializer(0), shape=conv_filter_shape[-1])

    # Convolution part
    conv_layer = tf.nn.conv2d(input_data, w, strides=conv_filter_stride, padding=padding)
    # conv_layer = tf.contrib.layers.batch_norm(conv_layer, is_training=is_training)

    conv_layer = conv_layer + bias
    conv_layer = tf.nn.relu(conv_layer)

    # Pooling part
    conv_layer = tf.nn.max_pool(conv_layer, 
                                ksize=pooling_filter_shape, 
                                strides=pooling_filter_stride, 
                                padding=padding)
    
    return conv_layer

## Build Conv Net Graph

In [ ]:
graph = tf.Graph()
with graph.as_default():
    with tf.name_scope('conv_image_net_inputs'):
        images = tf.placeholder(tf.float32, shape=[None, 784], name='images')
        labels = tf.placeholder(tf.float32, shape=[None, 10], name='labels')
        keep_dropout_prob = tf.placeholder(tf.float32, name='keep_dropout_prob')
        is_training = tf.placeholder(tf.bool, name='is_training')
            
    with tf.name_scope('image_reshape'):        
        images_reshaped = tf.reshape(images, [-1, 28, 28, 1])

    with tf.variable_scope('conv_layer_1'):
        conv_layer_1 = conv_layer('cl_1', images_reshaped, 
               conv_filter_shape=[3, 3, 1, 32], pooling_filter_shape=[1, 2, 2, 1], 
               conv_filter_stride=[1, 1, 1, 1], pooling_filter_stride=[1, 2, 2, 1],
               padding='SAME')  

    with tf.variable_scope('conv_layer_2'):
        conv_layer_2 = conv_layer('cl_2', conv_layer_1, 
               conv_filter_shape=[3, 3, 32, 64], pooling_filter_shape=[1, 2, 2, 1], 
               conv_filter_stride=[1, 1, 1, 1], pooling_filter_stride=[1, 2, 2, 1],
               padding='SAME') 
    
    with tf.variable_scope('feed_forward_layer_1'):
        ff_layer_1 = tf.reshape(conv_layer_2, [-1, 7*7*64])
        ff_layer_1 = simple_layer('ff_1', ff_layer_1, shape=[7*7*64, 10], activation='linear')
        raw_prediction = tf.nn.dropout(ff_layer_1, keep_dropout_prob)
    
    with tf.name_scope('prediction'):
        prediction = tf.nn.softmax(raw_prediction)

    with tf.name_scope('loss'):
        cross_entropy_vector = tf.nn.softmax_cross_entropy_with_logits(labels=labels, logits=raw_prediction)
        loss = tf.reduce_mean(cross_entropy_vector)
        
    with tf.name_scope('training'):
        # This dependency is here because of potential batch normalization and need of floating averages update
        update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
        with tf.control_dependencies(update_ops):
            train_step  = tf.train.AdamOptimizer(1e-3).minimize(loss)

    with tf.name_scope('accuracy'):
        correct_prediction = tf.equal(tf.argmax(prediction,1), tf.argmax(labels,1))
        correct_prediction = tf.cast(correct_prediction, tf.float32)
        accuracy = tf.reduce_mean(correct_prediction)
    
    initialize_vars = tf.global_variables_initializer()

## Run Conv Net

In [ ]:
validation_feed_dict = {
    images: data.validation.images, 
    labels: data.validation.labels,
    keep_dropout_prob: 1., 
    is_training: False}

session = tf.Session(graph=graph)
session.run(initialize_vars)

for iteration in range(5001):
    ##################
    # Training Phase #
    ##################
    
    _images, _labels = data.train.next_batch(100)
    feed_dict = {images: _images, labels: _labels, keep_dropout_prob: 0.5, is_training: True}
    _ = session.run([train_step], feed_dict=feed_dict)
 

    ####################
    # Validation Phase #
    ####################

    if iteration % 100 == 0:
        _accuracy, _loss = session.run([accuracy, loss], validation_feed_dict)
        print("= Valid Iteration {}: loss {}, accuracy {} =".format(iteration, _loss, _accuracy))

_prediction, = session.run([prediction], feed_dict=validation_feed_dict)

In [ ]:
session.close()

## Results evaluation

In [ ]:
from utils.results_evaluation import get_info_df
from utils.results_evaluation import get_accuracy
from utils.results_evaluation import get_false_positives
from utils.results_evaluation import get_info_df
from utils.results_evaluation import get_rec_prec
from utils.results_evaluation import plot_coocurance_matrix
from utils.results_evaluation import plot_examples 

In [ ]:
df = get_info_df(data.validation.labels, _prediction, class_id2class_name_mapping, data.validation.images)

In [ ]:
get_accuracy(df)

In [ ]:
get_rec_prec(df, class_id2class_name_mapping)

In [ ]:
fp = get_false_positives(df, 'Shirt')

In [ ]:
plot_examples(fp)

In [ ]:
plot_coocurance_matrix(df, use_top3=False, use_log=False)